# Linting by example

In this notebook, we demonstrate the creation of custom linting rules by supplying examples and counter-examples. While there are many great linting engines for Python already which capture a wide variety of code smells, creating custom, codebase-specific linting rules is a bit more difficult. 

For example, supposing we've just deprecated an old method for a newer, better version -- but, for legacy reasons, need to keep supporting the old one for now. We might want to alert developers, when linting, that they're continuing to use the deprecated method. We'll first supply examples of the deprecated method and preferred method:

In [1]:
from monkeys.asts import quoted
import ast


@quoted
def bad():
    my_obj.old_deprecated_method()
        
bad_ast = ast.Module(body=bad)


@quoted
def good():
    my_obj.new_great_method()
        
good_ast = ast.Module(body=good)

We can create queries against code structure with the external library [`astpath`](https://github.com/hchasestevens/astpath), which uses XPath expressions to query Python abstract syntax trees. Normally, using this means having a fairly good understanding of how Python ASTs look, but we can instead just have `monkeys` figure things out for us, given our examples.

We'll first off use the common XPath definitions available in `monkeys`, and register (for each of our examples) a suite of common syntax tree node names, attribute names, and attribute values.

In [2]:
from monkeys.typing import constant
from monkeys.common.xpath import NodeName, AttributeName, AttributeValue


class NodeRegistrar(ast.NodeVisitor):
    def visit(self, node):
        """Register AST node names and attributes with monkeys."""
        NodeName(node.__class__.__name__)
        
        fields = {
            field: getattr(node, field) 
            for field in 
            node._fields
        }
        for field, value in fields.items():
            if isinstance(value, ast.AST):
                NodeName(field)
                continue
                
            if isinstance(value, list):
                NodeName(field)
                for item in value:
                    if isinstance(item, str):
                        AttributeValue(item)
                continue
                
            AttributeName(field)
            if isinstance(value, str):
                AttributeValue(value)
                
        return super(NodeRegistrar, self).visit(node)


NodeRegistrar().visit(good_ast.body[0])
NodeRegistrar().visit(bad_ast.body[0])

We can now set up our scoring function, to try to find an `astpath` expression which matches on our deprecated method example, but not the new method example.

In [3]:
from monkeys.common.xpath import Expression
from monkeys.typing import params
from monkeys.search import optimize, assertions_as_score, pre_evaluate

from astpath import convert_to_xml, find_in_ast


good_xml = convert_to_xml(good_ast)
bad_xml = convert_to_xml(bad_ast)


@params(Expression)
@pre_evaluate
@assertions_as_score
def score(expression):
    assert expression.startswith('.//')  # as we want to capture this globally
    matches_good = bool(find_in_ast(good_xml, expression))
    matches_bad = bool(find_in_ast(bad_xml, expression))
    assert not matches_good
    assert matches_bad
    assert matches_good ^ matches_bad
    
    
best_expression = optimize(score).evaluate()

best_expression

Creating initial population of 250.
Optimizing...
Iteration 1:	Best: 2.00	Average: 1.00
Iteration 2:	Best: 2.00	Average: 1.00
Iteration 3:	Best: 2.00	Average: 2.00
Iteration 4:	Best: 2.00	Average: 1.00
Iteration 5:	Best: 4.00	Average: 1.00


".//Attribute[@attr = 'old_deprecated_method']//ctx"

Voila! We could now take this expression and run `astpath ".//Attribute[@attr = 'old_deprecated_method']//ctx"` against our codebase to find all deprecated method usages.